In [1]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import date
from datetime import timedelta

import dash
import dash_table
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

import plotly.graph_objs as go
import plotly.express as px

In [2]:
## Get Geolocations for federal states and districts
with open('C:/Users/simon/Jupyter/Covid/landkreise_simplify200.geojson', 'r') as read_file:
    geo_kreis = json.load(read_file)

with open('C:/Users/simon/Jupyter/Covid/bundeslaender_simplify200.geojson', 'r') as read_file:
    geo_land = json.load(read_file)

In [3]:
## Get population stats for federal states and districts
df_einwohner_kreis = pd.read_csv('C:/Users/simon/Jupyter/Covid/Einwohner_kreis.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})

df_einwohner_land = pd.read_csv('C:/Users/simon/Jupyter/Covid/Einwohner_land.csv', encoding = 'ISO-8859-1', sep=';', dtype= {'RS': str, 'Kreis': str, 'Einwohner': int})
df_einwohner_land.head()

,Bundesland,Einwohner
0,Baden-Württemberg,11100394
1,Bayern,13124737
2,Berlin,3669491
3,Brandenburg,2521893
4,Bremen,681202


In [4]:
## Get Covid stats from RKI
covid_url = 'https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.geojson'
r = requests.get(covid_url)
r.status_code

200

In [5]:
a = json.loads(r.content)
df = pd.json_normalize(a['features'])

df.head()

,type,geometry,properties.ObjectId,properties.IdBundesland,properties.Bundesland,properties.Landkreis,properties.Altersgruppe,properties.Geschlecht,properties.AnzahlFall,properties.AnzahlTodesfall,properties.Meldedatum,properties.IdLandkreis,properties.Datenstand,properties.NeuerFall,properties.NeuerTodesfall,properties.Refdatum,properties.NeuGenesen,properties.AnzahlGenesen,properties.IstErkrankungsbeginn,properties.Altersgruppe2
0,Feature,None,1,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-09-30T00:00:00Z,01001,"24.03.2021, 00:00 Uhr",0,-9,2020-09-30T00:00:00Z,0,1,0,Nicht übermittelt
1,Feature,None,2,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-10-29T00:00:00Z,01001,"24.03.2021, 00:00 Uhr",0,-9,2020-10-29T00:00:00Z,0,1,0,Nicht übermittelt
2,Feature,None,3,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-03T00:00:00Z,01001,"24.03.2021, 00:00 Uhr",0,-9,2020-11-03T00:00:00Z,0,1,0,Nicht übermittelt
3,Feature,None,4,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-20T00:00:00Z,01001,"24.03.2021, 00:00 Uhr",0,-9,2020-11-19T00:00:00Z,0,1,1,Nicht übermittelt
4,Feature,None,5,1,Schleswig-Holstein,SK Flensburg,A00-A04,M,1,0,2020-11-23T00:00:00Z,01001,"24.03.2021, 00:00 Uhr",0,-9,2020-11-18T00:00:00Z,0,1,1,Nicht übermittelt


In [6]:
## Clean-up and data preparation
df.columns = df.columns.str.replace('properties.', '')
df = df[['IdBundesland', 'Bundesland', 'Landkreis', 'IdLandkreis', 'AnzahlFall', 'Altersgruppe', 'AnzahlTodesfall', 'Geschlecht', 'Meldedatum']]
df['Meldedatum'] = df['Meldedatum'].str.replace('T00:00:00Z', '')
df['Meldedatum'] = pd.to_datetime(df['Meldedatum'])

df = df.merge(df_einwohner_kreis, how='left', left_on='IdLandkreis', right_on='RS')
df = df.merge(df_einwohner_land, how='left', left_on='Bundesland', right_on='Bundesland') 

df_gender = pd.get_dummies(df['Geschlecht']).mul(df['AnzahlFall'],0)
df_gender.columns = ['Sex_M', 'Sex_F', 'Sex_Unbekannt']
df = df.join(df_gender)

df_age = pd.get_dummies(df['Altersgruppe']).mul(df['AnzahlFall'],0)
df = df.join(df_age)



df.drop(['RS', 'Kreis'], axis=1, inplace=True)

df.head()

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-09-30,90164.0,...,1,0,0,1,0,0,0,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-10-29,90164.0,...,1,0,0,1,0,0,0,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-03,90164.0,...,1,0,0,1,0,0,0,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-20,90164.0,...,1,0,0,1,0,0,0,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-23,90164.0,...,1,0,0,1,0,0,0,0,0,0


In [7]:
df_bund = df.groupby(by=['Meldedatum']).agg({
    'AnzahlFall':'sum',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})


df_bund['Region'] = 'Deutschland'
df_bund.set_index('Region', append=True, inplace=True)
df_bund = df_bund.swaplevel('Region', 'Meldedatum')

df_bund['Fall_Last7'] = df_bund['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_bund['Fall_MA7'] = df_bund['AnzahlFall'].rolling(7).mean()
df_bund['R7'] = (df_bund['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_bund['Einwohner'] = df_einwohner_land['Einwohner'].sum()
df_bund['Fall_100K'] = ((df_bund['Fall_Last7'] / df_bund['Einwohner']) * 100000).round(2)


df_bund.tail()

AnzahlFall  AnzahlTodesfall  Sex_M  Sex_F  \
Region      Meldedatum                                              
Deutschland 2021-03-19       15525               18   7653   7720   
            2021-03-20       13042                6   6424   6522   
            2021-03-21        5860                5   3003   2821   
            2021-03-22        7790                9   3880   3765   
            2021-03-23       12603                8   6392   6056   

                        Sex_Unbekannt  A00-A04  A05-A14  A15-A34  A35-A59  \
Region      Meldedatum                                                      
Deutschland 2021-03-19            152      697     1610     4619     5822   
            2021-03-20             96      604     1353     3886     4861   
            2021-03-21             36      289      600     1690     2204   
            2021-03-22            145      394      862     2242     2919   
            2021-03-23            155      507     1202     3454     5165   

                        A60-A79  A80+  unbekannt  Fall_Last7      Fall_MA7  \
Region      Meldedatum                                                       
Deutschland 2021-03-19     2148   600         29     88300.0  12614.285714   
            2021-03-20     1871   449         18     90884.0  12983.428571   
            2021-03-21      836   238          3     91876.0  13125.142857   
            2021-03-22      999   351         23     92901.0  13271.571429   
            2021-03-23     1811   440         24     89760.0  12822.857143   

                          R7  Einwohner  Fall_100K  
Region      Meldedatum                              
Deutschland 2021-03-19  1.21   83166711     106.17  
            2021-03-20  1.17   83166711     109.28  
            2021-03-21  1.12   83166711     110.47  
            2021-03-22  1.09   83166711     111.70  
            2021-03-23  1.02   83166711     107.93

In [8]:
df_land = df.groupby(by=['Bundesland', 'Meldedatum']).agg({
    'IdBundesland': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_y': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})

df_land.rename(columns={'Einwohner_y': 'Einwohner'}, inplace=True)

df_land.index.names = ['Region', 'Meldedatum']

df_land['Fall_MA4'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_land['Fall_MA7'] = df_land.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_land['R4'] = df_land['Fall_MA4'].pct_change(periods=4) + 1
df_land['R7'] = (df_land['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_land['Fall_Last7'] = df_land['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_land['Fall_100K'] = ((df_land['Fall_Last7'] / df_land['Einwohner']) * 100000).round(2)

df_land.tail()

IdBundesland  AnzahlFall  Einwohner  AnzahlTodesfall  \
Region    Meldedatum                                                         
Thüringen 2021-03-19            16         836    2133378                2   
          2021-03-20            16         624    2133378                0   
          2021-03-21            16         299    2133378                0   
          2021-03-22            16         372    2133378                1   
          2021-03-23            16         697    2133378                0   

                      Sex_M  Sex_F  Sex_Unbekannt  A00-A04  A05-A14  A15-A34  \
Region    Meldedatum                                                           
Thüringen 2021-03-19    406    416             14       41       72      195   
          2021-03-20    309    304             11       21       62      132   
          2021-03-21    161    136              2       10       23       56   
          2021-03-22    185    181              6       13       46       85   
          2021-03-23    322    357             18       21       58      130   

                      A35-A59  A60-A79  A80+  unbekannt  Fall_MA4    Fall_MA7  \
Region    Meldedatum                                                            
Thüringen 2021-03-19      304      148    68          8    818.00  632.857143   
          2021-03-20      232      138    30          9    778.50  641.857143   
          2021-03-21      133       57    19          1    648.75  646.714286   
          2021-03-22      136       63    27          2    532.75  652.428571   
          2021-03-23      287      150    43          8    498.00  640.285714   

                            R4    R7  Fall_Last7  Fall_100K  
Region    Meldedatum                                         
Thüringen 2021-03-19  1.902326  1.19      4430.0     207.65  
          2021-03-20  1.605155  1.13      4493.0     210.60  
          2021-03-21  1.181156  1.13      4527.0     212.20  
          2021-03-22  0.769870  1.10      4567.0     214.07  
          2021-03-23  0.608802  1.01      4482.0     210.09

In [9]:
df_kreis = df.groupby(by=['Landkreis', 'Meldedatum']).agg({
    'IdLandkreis': 'first',
    'AnzahlFall': 'sum',
    'Einwohner_x': 'first',
    'AnzahlTodesfall': 'sum',
    'Sex_M': 'sum',
    'Sex_F': 'sum',
    'Sex_Unbekannt': 'sum',
    'A00-A04': 'sum',
    'A05-A14': 'sum',
    'A15-A34': 'sum',
    'A35-A59': 'sum',
    'A60-A79': 'sum',
    'A80+': 'sum',
    'unbekannt': 'sum'
})
    
df_kreis.rename(columns={'Einwohner_x': 'Einwohner'}, inplace=True)


df_kreis.index.names = ['Region', 'Meldedatum']

df_kreis['Fall_MA4'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(4).mean().values
df_kreis['Fall_MA7'] = df_kreis.groupby(level=0, group_keys=True)['AnzahlFall'].rolling(7).mean().values

df_kreis['R4'] = df_kreis['Fall_MA4'].pct_change(periods=4) + 1
df_kreis['R7'] = (df_kreis['Fall_MA7'].pct_change(periods=4) + 1).round(2)

df_kreis['Fall_Last7'] = df_kreis['AnzahlFall'].rolling(min_periods=1, window=7).sum()
df_kreis['Fall_100K'] = ((df_kreis['Fall_Last7'] / df_kreis['Einwohner']) * 100000).round(2)

df_kreis.tail()

IdLandkreis  AnzahlFall  Einwohner  \
Region             Meldedatum                                      
StadtRegion Aachen 2021-03-19       05334          96   557026.0   
                   2021-03-20       05334          71   557026.0   
                   2021-03-21       05334          11   557026.0   
                   2021-03-22       05334          36   557026.0   
                   2021-03-23       05334          71   557026.0   

                               AnzahlTodesfall  Sex_M  Sex_F  Sex_Unbekannt  \
Region             Meldedatum                                                 
StadtRegion Aachen 2021-03-19                0     48     48              0   
                   2021-03-20                1     33     38              0   
                   2021-03-21                0      4      7              0   
                   2021-03-22                0     14     21              1   
                   2021-03-23                0     43     28              0   

                               A00-A04  A05-A14  A15-A34  A35-A59  A60-A79  \
Region             Meldedatum                                                
StadtRegion Aachen 2021-03-19        2       10       35       43        6   
                   2021-03-20        1        4       27       28       10   
                   2021-03-21        1        0        4        3        3   
                   2021-03-22        2        4        8       17        5   
                   2021-03-23        4        4       23       27       12   

                               A80+  unbekannt  Fall_MA4   Fall_MA7        R4  \
Region             Meldedatum                                                   
StadtRegion Aachen 2021-03-19     0          0    121.75  83.714286  2.484694   
                   2021-03-20     1          0     98.00  82.857143  1.479245   
                   2021-03-21     0          0     67.00  83.000000  0.829721   
                   2021-03-22     0          0     53.50  86.428571  0.531017   
                   2021-03-23     1          0     47.25  72.857143  0.388090   

                                 R7  Fall_Last7  Fall_100K  
Region             Meldedatum                               
StadtRegion Aachen 2021-03-19  1.13       586.0     105.20  
                   2021-03-20  1.03       580.0     104.12  
                   2021-03-21  0.99       581.0     104.30  
                   2021-03-22  1.03       605.0     108.61  
                   2021-03-23  0.87       510.0      91.56

In [10]:
## Final dataset
df_merged = pd.concat([df_bund, df_land, df_kreis])
df_merged

## The geolocations for the districts of Berlin are not included in the dataset, 
## to have Berlin displayed on the choropleth map, I substitute the values for the federal state of Berlin.
df_merged.loc[df_merged.index.get_level_values('Region') == "Berlin", "IdLandkreis"] = "11000"

AnzahlFall  AnzahlTodesfall  Sex_M  Sex_F  \
Region             Meldedatum                                              
Deutschland        2020-01-07           1                0      1      0   
                   2020-01-19           1                0      0      1   
                   2020-01-23           1                0      1      0   
                   2020-01-25           1                0      0      1   
                   2020-01-28           2                0      2      0   
...                                   ...              ...    ...    ...   
StadtRegion Aachen 2021-03-19          96                0     48     48   
                   2021-03-20          71                1     33     38   
                   2021-03-21          11                0      4      7   
                   2021-03-22          36                0     14     21   
                   2021-03-23          71                0     43     28   

                               Sex_Unbekannt  A00-A04  A05-A14  A15-A34  \
Region             Meldedatum                                             
Deutschland        2020-01-07              0        0        0        0   
                   2020-01-19              0        0        0        1   
                   2020-01-23              0        0        0        0   
                   2020-01-25              0        0        0        0   
                   2020-01-28              0        0        0        1   
...                                      ...      ...      ...      ...   
StadtRegion Aachen 2021-03-19              0        2       10       35   
                   2021-03-20              0        1        4       27   
                   2021-03-21              0        1        0        4   
                   2021-03-22              1        2        4        8   
                   2021-03-23              0        4        4       23   

                               A35-A59  A60-A79  ...  unbekannt  Fall_Last7  \
Region             Meldedatum                    ...                          
Deutschland        2020-01-07        0        1  ...          0         1.0   
                   2020-01-19        0        0  ...          0         2.0   
                   2020-01-23        0        0  ...          0         3.0   
                   2020-01-25        1        0  ...          0         4.0   
                   2020-01-28        1        0  ...          0         6.0   
...                                ...      ...  ...        ...         ...   
StadtRegion Aachen 2021-03-19       43        6  ...          0       586.0   
                   2021-03-20       28       10  ...          0       580.0   
                   2021-03-21        3        3  ...          0       581.0   
                   2021-03-22       17        5  ...          0       605.0   
                   2021-03-23       27       12  ...          0       510.0   

                                Fall_MA7    R7   Einwohner  Fall_100K  \
Region             Meldedatum                                           
Deutschland        2020-01-07        NaN   NaN  83166711.0       0.00   
                   2020-01-19        NaN   NaN  83166711.0       0.00   
                   2020-01-23        NaN   NaN  83166711.0       0.00   
                   2020-01-25        NaN   NaN  83166711.0       0.00   
                   2020-01-28        NaN   NaN  83166711.0       0.01   
...                                  ...   ...         ...        ...   
StadtRegion Aachen 2021-03-19  83.714286  1.13    557026.0     105.20   
                   2021-03-20  82.857143  1.03    557026.0     104.12   
                   2021-03-21  83.000000  0.99    557026.0     104.30   
                   2021-03-22  86.428571  1.03    557026.0     108.61   
                   2021-03-23  72.857143  0.87    557026.0      91.56   

                               IdBundesland  Fall_MA4        R4  IdLandkreis  
Region   

In [103]:
app = dash.Dash(external_stylesheets=[dbc.themes.DARKLY])
server = app.server

In [104]:
colors = {
    'background': '#323130',
    'transparent': 'rgba(0,0,0,0)'
}

In [105]:
## Sunburst chart that seperates Covid cases into states first and districts second
def draw_sunburst():
    data = df[df['Meldedatum'] == df['Meldedatum'].max()]
    
    data = data.groupby(by=['Landkreis']).agg({
    'Bundesland': 'first',
    'AnzahlFall': 'sum',
    })
    data.reset_index(inplace = True)
    
    fig = px.sunburst(data, path=['Bundesland', 'Landkreis'], values='AnzahlFall')
    
    fig.update_layout(plot_bgcolor=colors['transparent'], 
                paper_bgcolor=colors['transparent'],
                font=dict(color='white'),
                margin=go.layout.Margin(l=0, r=0, t=20, b=20, pad=0))
    
    
    return fig

In [106]:
## Dropdown options for region-select
dropdown_options = []
for v in df_merged.index.unique(0).values:
    dropdown_options.append({
        'label': v, 'value': v
    })

In [107]:
tab_height = '3vh'
tab_style = {'padding': '0','line-height': tab_height, 'backgroundColor': 'gray', 'color': 'white', 'borderTop': '0px', 'borderBottom': '0px'}
tab_style_selected = {'padding': '0','line-height': tab_height, 'backgroundColor': 'dimgrey', 'color': 'white', 'borderTop': '0px', 'borderBottom': '3px solid royalblue'}
tab_style_selected_i = {'padding': '0','line-height': tab_height, 'backgroundColor': 'dimgrey', 'color': 'white', 'borderBottom': '0px', 'borderTop': '3px solid royalblue'}
border = '2px gray solid'

app.layout = html.Div([
    ## Title
    html.Div([
        html.Div([
            html.H4('COVID-19 in Germany'),
            html.P('This app queries the Robert-Koch-Institute Covid-19 dataset and displays relevant statistics in different graphics.')
        ], style={'width': '50%', 'float': 'left', 'display': 'inline-block'}),
        html.Div([
            html.H4('Last update'),
            html.H5(df['Meldedatum'].max().strftime('%d.%m.%Y'))
        ], style={'width': '20%', 'float': 'right', 'display': 'inline-block'})
    ], style={'width': '100%', 'float': 'left', 'display': 'inline-block', 'marginBottom': '30px'}),
    
    ## Content
    html.Div([
        ## Overview
        html.Div([
            html.Div([
                dcc.Graph(figure=draw_sunburst())
            ]),
            html.Div(style={'height': '20px'}),
            dcc.Tabs(id='scope-tabs', value='LAND', children=[
                dcc.Tab(label='Bundesländer', value='LAND', style=tab_style, selected_style=tab_style_selected_i),
                dcc.Tab(label='Landkreise', value='KREIS', style=tab_style, selected_style=tab_style_selected_i)
            ], style={'width': '50%','font-size': '100%', 'height':tab_height}),
            html.Div(id='scope-tabs-content'),
        ], style={'width': '40%', 'float': 'left', 'display': 'inline-block'}),
        ## Details
        html.Div([
            ## Region dropdown
            dcc.Dropdown(
                id='region-dropdown',
                options=dropdown_options,
                multi=False,
                value = 'Deutschland',
                clearable=False,
                style={'width': '80%', 'color':'dimgray'}
            ),
            ## Divider
            html.Div(style={'height': '30px'}),
            ## Quick Stats
            html.Div(id='quick-stats-content'),
            ## Divider
            html.Div(style={'height': '30px'}),
            ## General Statistics
            html.Div(id='stat-tabs-content', style={'border': border}),
            dcc.Tabs(id='stat-tabs', value='INCIDENCE', children=[
                dcc.Tab(label='Incidence', value='INCIDENCE', style=tab_style, selected_style=tab_style_selected),
                dcc.Tab(label='R Wert', value='R', style=tab_style, selected_style=tab_style_selected),
                dcc.Tab(label='Todesfälle', value='MORTALITY', style=tab_style, selected_style=tab_style_selected),
                dcc.Tab(label='Nightingale', value='NIGHT', style=tab_style, selected_style=tab_style_selected)
            ], style={'width': '55%','font-size': '100%', 'height':tab_height}),
            
            ## Divider
            html.Div(style={'height': '30px'}),
            
            ## Demographics
            html.Div(id='demographic-tabs-content', style={'border': border}),
            dcc.Tabs(id='demographic-tabs', value='AGE', children=[
                dcc.Tab(label='Alter', value='AGE', style=tab_style, selected_style=tab_style_selected),
                dcc.Tab(label='Alter in %', value='AGE_PCT', style=tab_style, selected_style=tab_style_selected),
                dcc.Tab(label='Geschlecht', value='SEX', style=tab_style, selected_style=tab_style_selected)
            ], style={'width': '50%','font-size': '100%', 'height':tab_height}),
        ], style={'width': '55%', 'float': 'right', 'display': 'inline-block'})
    ])

], style={ 'marginTop' : '20px',
            'marginRight' : '20px',
            'marginBottom' : '20px',
            'marginLeft' : '20px'})

In [108]:
## Displays quick stats about the selected region: todays number of cases and deaths, current incidence and R-value
@app.callback(Output('quick-stats-content', 'children'), Input('region-dropdown', 'value'))
def render_content(region):
    data = df_merged[df_merged.index.get_level_values('Region') == region].reset_index().iloc[-1]
    
    style_d = {
        'display': 'inline-block', 
        'marginTop' : '10px', 
        'marginRight' : '20px', 
        'marginBottom' : '10px', 
        'marginLeft' : '20px'
    }
    
    
    out = html.Div([
        html.Div([
            html.H4('New cases'),
            html.H5(data['AnzahlFall'])
        ], style=style_d),
        html.Div([
            html.H4('New deaths'),
            html.H5(data['AnzahlTodesfall'])
        ], style=style_d),
        html.Div([
            html.H4('Incidence'),
            html.H5(data['Fall_100K'])
        ], style=style_d),
        html.Div([
            html.H4('R'),
            html.H5(data['R7'])
        ], style=style_d),
    ], style={'border': border})
    
    return out

In [109]:
## Displays a colored map and a data-table with stats for either states or districts
def get_color(n):
    if(n < 35):
        return '<35'
    elif(n >= 35 and n < 50):
        return '35-50'
    elif(n >= 50 and n < 100 ):
        return '50-100'
    else:
        return '>100'

@app.callback(Output('scope-tabs-content', 'children'),
              Input('scope-tabs', 'value'))
def render_content(tab):
    
    data = df_merged.groupby(df_merged.index.get_level_values('Region')).last()
    data['Region'] = data.index.get_level_values('Region')
    
    if(tab == 'LAND'):
        data = data[data['IdBundesland'].notnull()]
        data['IdBundesland'] = data['IdBundesland'].astype(int).astype(str).str.zfill(2)
        geo = geo_land
        loc = 'IdBundesland'
    else:
        data = data[data['IdLandkreis'].notnull()]
        geo = geo_kreis
        loc= 'IdLandkreis'
    
    
    
    
    data['Inzidenz/100K'] = data['Fall_100K'].apply(get_color)

    fig_map = px.choropleth(data, geojson=geo, 
                           locations=loc, 
                           featureidkey='properties.RS', 
                           hover_name='Region',
                           hover_data=['Fall_100K', 'R7'],
                           basemap_visible=False,
                           fitbounds='locations',
                           color=data['Inzidenz/100K'],
                           color_discrete_map={'<35':'Green', '35-50':'Yellow','50-100':'Orange', '>100': 'Red'})
    
    fig_map.update_layout(plot_bgcolor=colors['transparent'], 
                          paper_bgcolor=colors['transparent'], 
                          geo_bgcolor=colors['transparent'],
                          font=dict(color='white'),
                          margin=go.layout.Margin(l=0, r=0, t=0, b=0, pad=0),
                          dragmode=False,
                          coloraxis_showscale=False)
    
    
    data_table = data[['Region', 'Fall_100K', 'R7', 'AnzahlFall', 'AnzahlTodesfall']]
    data_table.columns = ['Region', 'Inzidenz/100K', '7-Tages R', 'Neue Fälle', 'Neue Todesfälle']
    fig_table = dash_table.DataTable(
        columns = [{'name': col, 'id': col} for col in data_table.columns],
        data = data_table.to_dict(orient='records'),
        sort_action='native',
        page_action='native',
        page_current= 0,
        page_size= 16,
        style_as_list_view=True,
        style_header={'backgroundColor': 'rgb(30, 30, 30)'},
        style_cell={
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white'
    })

    
    return html.Div([
        dcc.Graph(id='r-map-chart', figure=fig_map),
        fig_table
    ], style={'border': border})
    

In [110]:
## Draws a nightingale chart showing the number of cured cases and deaths for each month, for the selected region
def draw_nightingale(data):
        data['Month'] = data.index.get_level_values('Meldedatum').strftime('%B %Y')
        data.reset_index(inplace=True)
        data = data.groupby(by=['Month']).agg({
            'AnzahlFall':'sum',
            'AnzahlTodesfall': 'sum',
            'Meldedatum': 'first'
        })
        data['Cured'] = data['AnzahlFall'] - data['AnzahlTodesfall']
        
        data.sort_values('Meldedatum', inplace=True, ascending=False)  
        data.reset_index(inplace=True)
        
        fig = go.Figure()
        fig.add_trace(go.Barpolar(
            r = list(data['Cured']),
            theta=list(data['Month']),
            name = 'Geheilt',
            marker_color='rgb(139,222,52)',
            marker_line_color='black',
            hoverinfo = ['all'],
            opacity=0.7   
        ))
        fig.add_trace(go.Barpolar(
            r = list(data['AnzahlTodesfall']),
            theta=list(data['Month']),
            name = 'Todesfälle',
            marker_color='rgb(245,94,21)',
            marker_line_color='black',
            hoverinfo = ['all'],
            opacity=0.7   
        ))
        
        
        fig.update_layout(
            
            font_size=12,
            legend_font_size=15,
            polar_angularaxis_rotation=90,
            polar = dict(
                      bgcolor = colors['transparent'],
                      angularaxis = 
                        dict(
                            linewidth = 2,
                            showline=True,
                            linecolor='gray'
                            ),
                  radialaxis = 
                        dict(        
                            showline = True,
                            linewidth = 2,
                            gridcolor = 'gray',
                            gridwidth = 2,
                            )
                        ),
                    )
        
        fig.update_xaxes(categoryorder='trace')
        
        return fig

In [111]:
## Shows general statistics for the selected region: Incidence, R-Value, mortality and a nightingale chart
@app.callback(Output('stat-tabs-content', 'children'),
              [Input('stat-tabs', 'value'), Input('region-dropdown', 'value')])
def render_content(tab, region):
    data = df_merged.loc[df_merged.index.get_level_values('Region') == region]
    
    if(tab=='INCIDENCE'):
        fig =  px.line(data, x=data.index.get_level_values('Meldedatum'), y='Fall_100K',
                      labels={'x': 'Meldedatum', 'Fall_100K': 'Inzidenz / 100.000'})

    elif(tab=='R'):
        r7_0 = data['R7'] - 1
        colors_i = np.where(r7_0 < 0, 'blue', 'red')
        
        fig = go.Figure(data=[
            go.Bar(x=data.index.get_level_values('Meldedatum'), y=r7_0,
              marker_color=colors_i, base=1,)
        ])
        
        fig.update_xaxes(title_text='Meldedatum')
        fig.update_yaxes(title_text='7-Tages R')
        
    elif(tab=='MORTALITY'):
        data['Mortality'] = data['AnzahlTodesfall'].cumsum()
        fig =  px.line(data, x=data.index.get_level_values('Meldedatum'), y='Mortality',
                      labels={'x': 'Meldedatum', 'Mortality': 'Kumulierte Todesfälle'})
    
    elif(tab=='NIGHT'):
        fig = draw_nightingale(data)
        
        
           
    fig.update_layout(plot_bgcolor=colors['transparent'], 
                          paper_bgcolor=colors['transparent'], 
                          font=dict(color='white'),
                          margin=go.layout.Margin(l=20, r=20, t=20, b=20, pad=20))
        
    return dcc.Graph(figure=fig)

In [112]:
## Draws a chart showing the proportion of cases for each age bracket
def draw_age_pct_chart(data):
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=data.index.get_level_values('Meldedatum'), 
            y=data['A00-A04'],
            name='00-04',
            mode='lines',
            line=dict(width=0.5, color='Green'),
            stackgroup='one',
            groupnorm='percent'
        ))
        fig.add_trace(go.Scatter(
            x=data.index.get_level_values('Meldedatum'), 
            y=data['A05-A14'],
            name='05-14',
            mode='lines',
            line=dict(width=0.5, color='Blue'),
            stackgroup='one'
        ))
        fig.add_trace(go.Scatter(
            x=data.index.get_level_values('Meldedatum'), 
            y=data['A15-A34'],
            name='05-14',
            mode='lines',
            line=dict(width=0.5, color='Purple'),
            stackgroup='one'
        ))
        fig.add_trace(go.Scatter(
            x=data.index.get_level_values('Meldedatum'), 
            y=data['A35-A59'],
            name='35-59',
            mode='lines',
            line=dict(width=0.5, color='Yellow'),
            stackgroup='one'
        ))
        fig.add_trace(go.Scatter(
            x=data.index.get_level_values('Meldedatum'), 
            y=data['A60-A79'],
            name='60-79',
            mode='lines',
            line=dict(width=0.5, color='Orange'),
            stackgroup='one'
        ))
        fig.add_trace(go.Scatter(
            x=data.index.get_level_values('Meldedatum'), 
            y=data['A80+'],
            name='80+',
            mode='lines',
            line=dict(width=0.5, color='Red'),
            stackgroup='one'
        ))
        fig.update_layout(
            showlegend=True,
            xaxis_type='date',
            yaxis=dict(
                type='linear',
                range=[1, 100],
                ticksuffix='%'))
        
        fig.update_xaxes(title_text='Meldedatum')
        fig.update_yaxes(title_text='Fälle nach Altersgruppe')
        
        return fig

In [113]:
## Shows demographic stats for the selected region: cases by age-group, proportion of age-groups and cases by gender
pd.set_option('mode.chained_assignment', None)
@app.callback(Output('demographic-tabs-content', 'children'),
              [Input('demographic-tabs', 'value'), Input('region-dropdown', 'value')])
def render_content(tab, region):
    data = df_merged.loc[df_merged.index.get_level_values('Region') == region]
    
    if(tab == 'AGE'):
        data[['00-04', '05-14', '15-34', '35-59', '60-79', '80+']] = data[['A00-A04', 'A05-A14', 'A15-A34', 'A35-A59', 'A60-A79', 'A80+']].rolling(7).mean().values
        fig = px.line(data, x=data.index.get_level_values('Meldedatum'), y=['00-04', '05-14', '15-34', '35-59', '60-79', '80+'], 
                     labels={'x': 'Meldedatum', 'value': 'Inzidenz / 100.000', 'variable': 'Altersgruppe'})
        
    if(tab=='AGE_PCT'):
        fig = draw_age_pct_chart(data)
    
    elif(tab == 'SEX'):
        fig = px.bar(data, x=['Male', 'Female'], y=[data['Sex_M'].sum(), data['Sex_F'].sum()],
                    labels={'x': 'Geschlecht', 'y': 'Fälle (Gesamt)'})
        
    
    fig.update_layout(plot_bgcolor=colors['transparent'], 
                        paper_bgcolor=colors['transparent'], 
                        font=dict(color='white'),
                        margin=go.layout.Margin(l=20, r=20, t=20, b=20, pad=20))
    
    return dcc.Graph(figure=fig)

In [114]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [83]:
df.head()

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
0,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-09-30,90164.0,...,1,0,0,1,0,0,0,0,0,0
1,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-10-29,90164.0,...,1,0,0,1,0,0,0,0,0,0
2,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-03,90164.0,...,1,0,0,1,0,0,0,0,0,0
3,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-20,90164.0,...,1,0,0,1,0,0,0,0,0,0
4,1,Schleswig-Holstein,SK Flensburg,01001,1,A00-A04,0,M,2020-11-23,90164.0,...,1,0,0,1,0,0,0,0,0,0


In [84]:
df[df["Landkreis"].str.contains("Berlin")]

,IdBundesland,Bundesland,Landkreis,IdLandkreis,AnzahlFall,Altersgruppe,AnzahlTodesfall,Geschlecht,Meldedatum,Einwohner_x,...,Sex_M,Sex_F,Sex_Unbekannt,A00-A04,A05-A14,A15-A34,A35-A59,A60-A79,A80+,unbekannt
1142891,11,Berlin,SK Berlin Mitte,11001,1,A05-A14,0,M,2020-12-16,NaN,...,1,0,0,0,1,0,0,0,0,0
1143008,11,Berlin,SK Berlin Mitte,11001,4,A05-A14,0,M,2020-12-16,NaN,...,4,0,0,0,4,0,0,0,0,0
1143011,11,Berlin,SK Berlin Mitte,11001,1,A05-A14,0,M,2020-12-17,NaN,...,1,0,0,0,1,0,0,0,0,0
1143014,11,Berlin,SK Berlin Mitte,11001,1,A05-A14,0,M,2020-12-17,NaN,...,1,0,0,0,1,0,0,0,0,0
1143512,11,Berlin,SK Berlin Mitte,11001,2,A05-A14,0,M,2020-12-18,NaN,...,2,0,0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217495,11,Berlin,SK Berlin Reinickendorf,11012,1,A80+,1,M,2020-12-28,NaN,...,1,0,0,0,0,0,0,0,1,0
1217496,11,Berlin,SK Berlin Reinickendorf,11012,3,A80+,0,M,2020-12-28,NaN,...,3,0,0,0,0,0,0,0,3,0
1217497,11,Berlin,SK Berlin Reinickendorf,11012,1,A80+,0,M,2020-12-28,NaN,...,1,0,0,0,0,0,0,0,1,0
1217498,11,Berlin,SK Berlin Reinickendorf,11012,1,A80+,0,M,2020-12-29,NaN,...,1,0,0,0,0,0,0,0,1,0


In [101]:
df_merged.loc[df_merged.index.get_level_values('Region') == "Berlin", "IdLandkreis"] = "11000"

In [102]:
df_merged.loc[df_merged.index.get_level_values('Region') == "Berlin", "IdLandkreis"]

Region  Meldedatum
Berlin  2020-01-19    11000
        2020-03-03    11000
        2020-03-04    11000
        2020-03-05    11000
        2020-03-06    11000
                      ...  
        2021-03-19    11000
        2021-03-20    11000
        2021-03-21    11000
        2021-03-22    11000
        2021-03-23    11000
Name: IdLandkreis, Length: 385, dtype: object